In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
import datetime
from datetime import date, timedelta
import seaborn as sns
import time


from fund_tools import *

In [2]:
# Set date variable

today = date.today()
first_day_of_the_week = today - timedelta(days=today.weekday())
# end = start + timedelta(days=6)
print("Today: " + str(today))
print("Start: " + str(first_day_of_the_week))
# print("End: " + str(end))

Today: 2021-09-30
Start: 2021-09-27


In [3]:
start_date = '2015-01-01'
end_date = today

fund_net_value = get_fund_net_worth('001980',start_date=start_date, end_date=end_date, fund_category='open')

In [182]:
def AIP_weekly(code, start_date, end_date, fund_category, fixed_investment, freq='Monday', df=False):
    
    
    fund_net_value = get_fund_net_worth(code, start_date=start_date, end_date=end_date, fund_category=fund_category)
    
    fund_net_value['WeekDay'] = pd.to_datetime(fund_net_value['净值日期']).dt.day_name()
    
    fund_net_value['定投金额(本金)'] = 0
        
    for i in range(len(fund_net_value['WeekDay'])):
         if fund_net_value['WeekDay'].values[i] == freq:
                fund_net_value['定投金额(本金)'][i] = fixed_investment
            
    fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
    fund_net_value['购买份额'] = fund_net_value['定投金额(本金)']/fund_net_value['单位净值']
    fund_net_value['累计份额'] = fund_net_value['购买份额'].cumsum()
    fund_net_value['平均成本'] = fund_net_value['累计定投金额(本金)']/fund_net_value['累计份额']

    fund_net_value['累计收益'] = (fund_net_value['单位净值'] - fund_net_value['平均成本']) * fund_net_value['累计份额']
    
    start_invest = fund_net_value['定投金额(本金)'].values.nonzero()[0][0]
    fund_net_value['持有天数'] = (fund_net_value['净值日期'] - fund_net_value['净值日期'][start_invest]).dt.days+1
    for i in range(len(fund_net_value['持有天数'])):
        if fund_net_value['持有天数'][i] < 0:
            fund_net_value['持有天数'][i] = 0
    fund_net_value['年化收益率'] = ((fund_net_value['累计收益'] + fund_net_value['累计定投金额(本金)'])/fund_net_value['累计定投金额(本金)'])**(365/fund_net_value['持有天数'])-1

    fund_net_value['累计收益率'] = fund_net_value['累计收益']/fund_net_value['累计定投金额(本金)']
    
    Stat_df = pd.DataFrame({
        '基金代码': code,
        '持有天数': fund_net_value['持有天数'].values[-1],
        '定投时间': freq,
        '定投金额': fixed_investment,
        '分投期数': fund_net_value['累计定投金额(本金)'].values[-1]/fixed_investment,
        '总购买份额' : '%.3f' % fund_net_value['累计份额'].values[-1],
        '平均成本' : '%.3f' % fund_net_value['平均成本'].values[-1],
        '累计收益' : '%.3f' % fund_net_value['累计收益'].values[-1],
        '累计收益率' : '%.3f' % fund_net_value['累计收益率'].values[-1],
        '年化收益率' : '%.3f' % fund_net_value['年化收益率'].values[-1]
    }, index=['Plan'])
    
    if df:
        return fund_net_value
    else:
        return Stat_df

In [184]:
AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, freq='Monday', df=False)

,基金代码,持有天数,定投时间,定投金额,分投期数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Plan,001980,1194,Monday,1000,158.0,138466.734,1.141,106596.083,0.675,0.171


In [189]:
def AIP_Weekly_Plans(Freq, code, start_date, end_date, fund_category, fixed_investment, df=False):
    
    df = pd.DataFrame()
    
    for freq in Freq:
        df = df.append(AIP_weekly(code, start_date=start_date, end_date=end_date, fund_category=fund_category, fixed_investment=fixed_investment, freq=freq, df=False))
        
    return df

In [192]:
Freq = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

AIP_Weekly_Plans(Freq, '001980', start_date, end_date, fund_category='open', fixed_investment=1000, df=False)

,基金代码,持有天数,定投时间,定投金额,分投期数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Plan,001980,1194,Monday,1000,158.0,138466.734,1.141,106596.083,0.675,0.171
Plan,001980,1200,Tuesday,1000,162.0,141338.483,1.146,108083.707,0.667,0.168
Plan,001980,1234,Wednesday,1000,164.0,143250.605,1.145,109737.582,0.669,0.164
Plan,001980,1205,Thursday,1000,164.0,143136.130,1.146,109518.831,0.668,0.168
Plan,001980,1232,Friday,1000,162.0,141864.343,1.142,109088.573,0.673,0.165
